<h4>-----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.</h4>
<h5> THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN <br> THE  SOFTWARE.<br>-----------------------------------------------------------------------------</h5>

In [ ]:
import time

from arena_api.__future__.save import Recorder
from arena_api.buffer import BufferFactory
from arena_api.enums import PixelFormat
from arena_api.system import system

#### Save Recorder: Introduction
>This example demonstrates creating a save recorder to save videos from image buffer data. This includes, configuring and initializing save recorder, getting and appending buffers to the recorder, saving the video.

In [ ]:
"""
Waits for the user to connect a device before raising
an exception if it fails
"""
tries = 0
tries_max = 6
sleep_time_secs = 10
devices = None
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                    '.' * sec_count, end='\r')
        tries += 1
    else:
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

#### Setup stream nodes

In [ ]:
device = devices[0]
nodemap = device.nodemap
tl_stream_nodemap = device.tl_stream_nodemap
print(f'Device used in the example:\n\t{device}')

# Enable stream auto negotiate packet size
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

# Enable stream packet resend
tl_stream_nodemap['StreamPacketResendEnable'].value = True

In [ ]:
"""
Configure camera nodes
"""
"""
set width and height to max values might make the video frame rate low
The larger the height of the buffer the lower the fps
"""
width_node = nodemap['Width']
width = nodemap['Width'].max

height_node = nodemap['Height']
height = nodemap['Height'].max

"""
if the images from the device are already in the format expected by the
recorder then no need to convert received buffers which results in better
performance
"""
nodemap['PixelFormat'].value = PixelFormat.BGR8

#### demonstrates Save Recorder
1. Setup stream nodes
2. Configure device nodes
3. Create a recorder object and configure its width, height, and acquisition frame rate
4. Open recorder, start stream and get buffers
5. Append buffers to the recorder
6. Close the recorder to save the video
    

In [ ]:
with device.start_stream(100):
    print('Stream started')

    """
    create a recorder
    The recorder, takes width, height, and frames per seconds.
    These argument can be deferred until Recorder.open is called
    """
    recorder = Recorder(nodemap['Width'].value,
                        nodemap['Height'].value,
                        nodemap['AcquisitionFrameRate'].value)
    print('fps',  nodemap['AcquisitionFrameRate'].value)

    recorder.codec = ('h264', 'mp4', 'bgr8')  # order does not matter

    """
    recorder settings can not be changed after open is called util
    close is called
    """
    recorder.open()
    print('recorder opened')

    TOTAL_IMAGES = 100
    for count in range(TOTAL_IMAGES):
        buffer = device.get_buffer()
        print(f'Image buffer received')

        """
        After recorder.open() add image to the open recorder stream by
            appending buffers to the video.
        The buffers are already BGR8, because we set 'PixelFormat'
            node to 'BGR8', so no need to convert buffers using
            BufferFactory.convert() from arena_api.buffer
        """

        """
        default name for the video is 'video<count>.mp4' where count
        is a pre-defined tag that gets updated every time open()
        is called. More custom tags can be added using
        Recorder.register_tag() function
        """
        recorder.append(buffer)
        print(f'Image buffer {count} appended to video')

        device.requeue_buffer(buffer)
        print(f'Image buffer requeued')

    recorder.close()
    print('recorder closed')
    print(f'video saved {recorder.saved_videos[-1]}')

    video_length_in_secs = (TOTAL_IMAGES /
                            nodemap['AcquisitionFrameRate'].value)
    print(f'video length is {video_length_in_secs} seconds')

In [ ]:
system.destroy_device()
print('Destroyed all created devices')